As mentioned [here](http://manuelpm.me/VIMOSlabbook16/log/2017/05/16/cx-359,644,950-and-953.html), in the same folder as [CX359](https://vimos.manuelpm.me/cx359) in the first section of the SEXM there is CX953.

- CX953: 0 to 50. There are at least two sources. One brighter thought the whole spectral range center at around 30. The second one center around 17.

In [16]:
#Import packages
from astropy.io import fits
import os
from stsci.tools import capable
capable.OF_GRAPHICS = False
from pyraf import iraf
import numpy as np
from shutil import copyfile
from astropy.convolution import convolve, Box1DKernel
#Bokeh plotting
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import HoverTool, tools, ColumnDataSource, CustomJS, Slider, BoxAnnotation
output_notebook()

Loading BokehJS ...

First we define the parameters to extract the desire spectra:
    - Name of the source
    - Center, low and upper limits of the aperture
    - background sample

In [111]:
sourcename = 'cx953sky'
center = 31
low = -5
high = 5
b_low = [-10,-6]
b_up = [10,6]
databasegeneral = 'database/apcx359sexm'
filename = 'database/ap'+sourcename
fitsfilename = 'mos_science_sky_extracted_Q3.fits'

First we multiply the 2D spectra times the exposure time

In [18]:
spectrawithsky = fits.open(fitsfilename)
exptime = spectrawithsky[0].header['EXPTIME']
iraf.stsdas()
iraf.images.imutil()
if os.path.exists(sourcename+'.fits'):
    os.remove(sourcename+'.fits')
    

iraf.images.imutil.imarith(fitsfilename,'*', exptime,sourcename)

Now we need to change the database of the aperture to define the center. This is the only way I have found to do it automatically. After this define the parameters in the .par file

In [108]:
copyfile(databasegeneral,filename)
#Now read and replace center and upper and lower values
with open(filename) as f:
    for lines in f:
        if 'image' in lines:
                imagename= lines.split()[1]
        if 'center' in lines:
                numerocenter = lines.split()[2]
        if 'low' in lines:
                numerolow = lines.split()[2]
        if 'high' in lines:
                numerohigh = lines.split()[2]
        if 'xmin' in lines:
            bloworiginal  = lines
            blow = lines.replace(lines.split()[1],min(b_low)
        if 'xmax' in lines:
            buporiginal  = lines
            bup = lines.replace(lines.split()[1],max(b_up)
        if 'sample' in lines:
            sampleor = lines
            sampleb = lines.replace(-10,min(b_low))
                            
            
                
                
                break

with open(filename) as f:
    filedata = f.read()

filedata = filedata.replace(imagename,sourcename)
filedata = filedata.replace(numerocenter,str(center))
filedata = filedata.replace(numerolow,str(low))
filedata = filedata.replace(numerohigh,str(high))

with open(filename,'w') as f:
    f.write(filedata)



In [117]:
with open(filename) as f:
    for lines in f:
        if 'sample' in lines:
            a = lines
            b = lines.replace('-10','1')
            print b.split()[1]
filedata = filedata.replace(a,b)

with open(filename,'w') as f:
    f.write(filedata)




1:-6,6:10


In [75]:
if os.path.exists(sourcename+'.ms.fits'):
    os.remove(sourcename+'.ms.fits')

#Call them 
iraf.noao.twodspec()
iraf.noao.twodspec.apextract()
iraf.noao.twodspec.apextract.setParam('dispaxis','1')
#http://vivaldi.ll.iac.es/sieinvens/siepedia/pmwiki.php?n=HOWTOs.PythonianIRAF
iraf.noao.apextract.apall.setParam('input',sourcename+'.fits')
#try output
iraf.noao.apextract.apall.setParam('output',sourcename+'.ms.fits')
iraf.noao.twodspec.apextract.apall.setParam('recenter','no')
iraf.noao.twodspec.apextract.apall.setParam('resize','no')
iraf.noao.twodspec.apextract.apall.setParam('edit','no')
iraf.noao.twodspec.apextract.apall.setParam('trace','no')
iraf.noao.twodspec.apextract.apall.setParam('interactive','no')
iraf.noao.twodspec.apextract.apall.setParam('apertures','1')
iraf.noao.twodspec.apextract.apall.setParam('find','no')
iraf.noao.twodspec.apextract.apall.setParam('clean','yes')
iraf.noao.twodspec.apextract.apall.setParam('background','average')
iraf.noao.twodspec.apextract.apall.setParam('b_sample','-10,0:0,0')
iraf.noao.apextract.apall.saveParList(filename='uparm/'+sourcename+'.par')
iraf.noao.twodspec.apextract.apall(ParList='uparm/'+sourcename+'.par')

May 23 17:29: EXTRACT - No apertures defined for cx953sky


In [110]:
cat database/apcx953sky

# Tue 01:28:30 14-Mar-2017
begin	aperture cx953sky 1 1250. 31
	image	cx953sky
	aperture	1
	beam	1
	center	1250. 31
	low	-1249. -5
	high	1250. 5
	background
		xmin 1.
		xmax 10.
		function chebyshev
		order 1
		sample -10:-6,6:10
		naverage -3
		niterate 0
		low_reject 3.
		high_reject 3.
		grow 0.
	axis	2
	curve	5
		2.
		1.
		1.
		2500.
		0.



In [52]:
#Plotting
##For srfm[0].header["CTYPE1"] = 'LINEAR'
srfm = fits.open(sourcename+'.ms.fits')
#secondstar = srfm[0].data#[0][0]##[0][0] if using clean
secondstar = srfm[0].data[0][0]
xn = srfm[0].header["NAXIS1"]
refx = srfm[0].header["CRVAL1"]
step = srfm[0].header['CD1_1']
cr = srfm[0].header['CRPIX1']
#
xlist = [ refx + step*(i - cr) for i in np.arange(1, len(secondstar)+1) ]
#
hover = HoverTool(
        tooltips=[
            #("index", "$index"),
            ("(x,y)", "($x{1}, $y)"),
        ]
    )

#Create ColumnDataSource
x = np.array(xlist)
y = np.array(secondstar)
source = ColumnDataSource(data=dict(x=x,y=y))




plot = figure(x_axis_label='Angstrom', y_axis_label='Y',title="Spectra")
plot.add_tools(hover)
plot.add_tools(tools.ResizeTool())
#Eraaseplot.line(xlist,secondstar)
plot.line('x','y',source=source)
show(plot)

# Tue 01:28:30 14-Mar-2017
begin	aperture cx953sky 1 1250. 31
	image	cx953sky
	aperture	1
	beam	1
	center	1250. 31
	low	-1249. -5
	high	1250. 5
	background
		xmin -10.
		xmax 10.
		function chebyshev
		order 1
		sample -10:-6,6:10
		naverage -3
		niterate 0
		low_reject 3.
		high_reject 3.
		grow 0.
	axis	2
	curve	5
		2.
		1.
		1.
		2500.
		0.



In [120]:
min(b_low)

-10

In [131]:
a.isdigit

<function isdigit>

In [153]:
a[14] == '1'


False

In [138]:
c=[]
for i in a:
    if i.isdigit():
        c.append(i)

In [172]:
for i,j in enumerate(c):
    t= a.replace(j,str(bb[i]),1)
print t
    

		sample 1:-6,6:15



In [170]:
a


'\t\tsample 1:-6,6:10\n'

In [168]:
bb=[1,2,3,4,5]